In [15]:
import pandas as pd 
import numpy 
import spacy 
import nltk 
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en.stop_words import STOP_WORDS
import re
from nltk.tokenize import word_tokenize

import string 
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
"""neural networks libraries"""
import tensorflow as tf
from keras.models import Sequential
from keras.layers import InputLayer,BatchNormalization,Flatten,Embedding,LSTM,Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.activations import softmax
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils

""" sklearns libraries """

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import accuracy_score,classification_report
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [3]:
data=pd.read_csv("one_data",usecols=('text','target'))

data.head()

,text,target
0,i didnt feel humiliated,7
1,i can go from feeling so hopeless to so damned...,7
2,im grabbing a minute to post i feel greedy wrong,2
3,i am ever feeling nostalgic about the fireplac...,4
4,i am feeling grouchy,2


In [4]:
data.dropna(inplace=True)

# Text Preprocessesing

In [5]:
def llematization(text):
    lm=WordNetLemmatizer()
    text=text.split()
    text=[lm.lemmatize(y) for y in text]
    return ' '.join(text)
def rm_stopwords(text):
    english_stopword=set(stopwords.words('english')).union(set(STOP_WORDS))
    text=[i for i in str(text).split() if i not in english_stopword ]
    return ' '.join(text)
def lower_case(text):
    text=text.split()
    text=[y.lower() for y in text ]
    return' '.join(text)
def select_only_char(text):
    text=re.sub(r'[^a-z]', ' ', text)
    text=' '.join(text.split())
    return text.strip()
def normalize_text(data):
    data.text=data.text.apply(lambda text : llematization(text))
    data.text=data.text.apply(lambda text : rm_stopwords(text))
    data.text=data.text.apply(lambda text : lower_case(text))
    data.text=data.text.apply(lambda text : select_only_char(text))
    return data

In [6]:

data=normalize_text(data.dropna())

In [7]:

data

,text,target
0,didnt feel humiliated,7
1,feeling hopeless damned hopeful care awake,7
2,im grabbing minute post feel greedy wrong,2
3,feeling nostalgic fireplace know property,4
4,feeling grouchy,2
...,...,...
110582,my friend i took trip hampton basslights booke...,4
110583,check departure staff friendly professional wa...,5
110584,this hampton located quiet street hospital loc...,5
110585,awesome wing my favorite wa garlic parmesan gr...,5


In [8]:
x=data['text']
y=data['target']

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.1, random_state =42,stratify=y)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((99508,), (11057,), (99508,), (11057,))

In [10]:
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=40000)
vectoriser.fit(x_train)
print('No. of feature_words: ', len(vectoriser.get_feature_names()))

No. of feature_words:  40000


C:\Users\kabir\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [11]:
X_train = vectoriser.transform(x_train)
X_train

<99508x40000 sparse matrix of type '<class 'numpy.float64'>'
	with 2647581 stored elements in Compressed Sparse Row format>

In [12]:
X_test  = vectoriser.transform(x_test)
X_test

<11057x40000 sparse matrix of type '<class 'numpy.float64'>'
	with 289795 stored elements in Compressed Sparse Row format>

In [13]:
def model_Evaluate(model):
    # Predict values for Test dataset
    y_pred = model.predict(X_test)
    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)
    categories = ['Negative','Positive']
    group_names = ['True Neg','False Pos', 'False Neg','True Pos']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]
    labels = [f'{v1} : {v2}' for v1, v2 in zip(group_names,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
    xticklabels = categories, yticklabels = categories)
    plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual values" , fontdict = {'size':14}, labelpad = 10)
    plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)

In [16]:
lg=LogisticRegression()
lg.fit(X_train,y_train)
#model_Evaluate(lg)
y_pred1=lg.predict(X_test)
classification_report(y_pred1,y_test)

C:\Users\kabir\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


'              precision    recall  f1-score   support\n\n           1       0.53      0.59      0.56       485\n           2       0.39      0.68      0.50       402\n           3       0.45      0.55      0.50      1212\n           4       0.59      0.60      0.60      2978\n           5       0.73      0.67      0.70      3067\n           6       0.52      0.71      0.60       166\n           7       0.81      0.67      0.73      2747\n\n    accuracy                           0.64     11057\n   macro avg       0.58      0.64      0.60     11057\nweighted avg       0.66      0.64      0.64     11057\n'